# Number Array Machine Learning Project

## Module Imports

In [ ]:
!pip install imbalanced-learn

In [ ]:
import numpy as np
import pandas as pd
import itertools
import random

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


## Class with all Methods

In [ ]:
class NumberArray():
  """
  It contain all the methods used in this project.

  Methods:
  - create_dataset => Method for creating dataset
  - label_dataset => Method for labeling the dataset
  - label_sample => Method returns the sample of the provided sample
  """

  def __init__(self, n, k):
    """
    Constructor of the class. Maps the provided arguments to respective
    attributes. 
    """
    
    self.n = n
    self.k = k

  def create_dataset(self):
    """
    Uses n and k class parameters and create an array with all possible 
    combinations with repetition. It converts this array into a pandas dataframe 
    for dealing with data in easy and better way.

    Dataframe containing all data is return when this method is called. 
    """

    arr = [i for i in range(1, self.n + 1)]
    all_combinations = np.array(list(itertools.product(arr, repeat = self.k)))
    self.dataset = pd.DataFrame(all_combinations, columns=range(1, self.k + 1))
    return self.dataset

  def create_random_sample_dataset(self, no_of_samples = 1000):
    """
    This method creates create random samples of the permutation. It takes no_of_samples
    in argument and create a dataset with that number of random samples. Default no_of_samples 
    is set to 1000.
    """
    
    arr = [i for i in range(1, self.n + 1)]
    combinations = set()
    temp = len(combinations)
    working_fine = no_of_samples

    for10 = self.n
    while for10 >= 0.6 * self.n :
        arr10 = [random.choice([1,2]) for i in range(2,self.k+1)] + [for10]
        for i in range(25):
          combinations.add(tuple(np.random.choice(arr10, self.k, replace=False)))
          combinations.add(tuple(np.random.choice([1 for i in range(1, self.k+1)], self.k, replace=False)))
        for10 -= 1

    while len(combinations) != no_of_samples and working_fine:
      combinations.add(tuple(np.random.choice(arr, self.k)))
      
      if len(combinations) == temp:
        working_fine -= 1
      else:
        temp = len(combinations)
        working_fine = no_of_samples

    if not working_fine:
      print(f"{no_of_samples} samples are not possible. Max permutation for our case are {len(combinations)}")

    print(combinations)
    combinations_tuple = tuple(combinations)
    self.dataset = pd.DataFrame(combinations_tuple, columns=range(1, self.k + 1))
    return self.dataset

  def label_dataset(self):
    """
    It creates a labeled dataset. Copies the dataset so that changes doesnot 
    effect original dataset. Create a new column for output labels and returns
    the dataset with labels.
    """
    
    self.labeled_dataset = self.dataset.copy()
    self.labeled_dataset['unique'] = self.labeled_dataset.apply(
        lambda sample: self.unique(sample.values), axis=1
        )
    self.labeled_dataset['difference_max'] = self.labeled_dataset.apply(
        lambda sample: self.difference_max(sample.values), axis=1
        )
    self.labeled_dataset['difference_last'] = self.labeled_dataset.apply(
        lambda sample: self.difference_last(sample.values), axis=1
        )
    self.labeled_dataset['output'] = self.labeled_dataset.apply(
        lambda sample: self.label_sample(sample), axis=1
        )
    return self.labeled_dataset
  
  def unique(self, sample):
    """
    Method to return unique value for creating a new feature of the dataset
    """
    sample = sample[:self.k]
    unique = len(list(np.unique(sample)))
    return unique

  def difference_max(self, sample):
    """
    Method to return difference between maximum and minimum value of the sample for creating a new feature of the dataset
    """
    sample = sample[:self.k]
    unique = list(np.unique(sample))
    return max(unique) - min(unique)
  
  def difference_last(self, sample):
    """
    Method to return difference between maximum and second largest value of the sample for creating a new feature of the dataset
    """
    
    sample = sample[:self.k]
    unique = sorted(list(np.unique(sample)))
    new_unique = sorted(list(np.unique(np.array(sorted(list(sample))[:-1]))))
    return max(unique) - max(new_unique)
  
  def label_sample(self, sample):
    """
    Return labels of each sample provided.
    """
    
    sample_value = sample.values
    sample_value = sample_value[:self.k]
    unique = list(np.unique(sample_value))
    if sorted(unique) == list(range(min(unique), max(unique) + 1)):
      if len(unique) <= 0.6 * self.k:
        return 30
      return 20
    else:
      new_sample_value = list(sorted(sample_value)[:-1])
      new_unique = list(np.unique(new_sample_value))
      if sorted(new_unique) == list(range(min(new_unique), max(new_unique) + 1)):
        if max(unique) - max(new_unique) >= 0.70 * self.n:
          if len(unique) <= 0.6 * self.k:
            return 10
      return 20
  
  def train_model(self, test_size = 0.3 ):
    """
    Method for training a Naive Bayes Model and predict the output using provided features.
    
    It returns a dataframe with all the features, i.e, engineered features, input features, output label and predicted label.
    """
    
    labeled_dataset_copied = self.labeled_dataset.copy()
    X = labeled_dataset_copied.drop(["output"], axis=1).values
    y = self.labeled_dataset["output"].values

    oversample =SMOTE(k_neighbors=2)
    X, y = oversample.fit_resample(X, y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0)
    
    #self.guassian_nb = GaussianNB()
    #self.guassian_nb.fit(X_train, y_train)
    #y_pred = self.guassian_nb.predict(X_test)
    
    #self.clf = RandomForestClassifier(max_depth=2, random_state=0)
    self.clf = RandomForestClassifier(random_state=0)   
    self.clf.fit(X_train, y_train)
    y_pred = self.clf.predict(X_test)

    print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != y_pred).sum()))

    print('\033[1m'+"Accuracy of the Output is: " +'\033[91m'+ str(accuracy_score(y_test, y_pred)*100) + "%"+"\033[0;0m")

    columns = [i for i in range(1, self.k + 1)] + ["unique", "difference_max", "difference_last"]
    
    self.predicted_dataset = pd.DataFrame(X_test, columns=columns)
    self.predicted_dataset["Original Output"] = y_test
    self.predicted_dataset["Predicted Output"] = y_pred
    
    return self.predicted_dataset

  def predict_sample(self, sample):
    """
    Method for predicting a single sample of the dataset.
    """
    
    unique = self.unique(np.array(sample))
    difference_max = self.difference_max(np.array(sample))
    difference_last = self.difference_last(np.array(sample))

    sample.append(unique)
    sample.append(difference_max)
    sample.append(difference_last)

    pred_sample = np.array([sample])
    prediction = self.clf.predict(pred_sample)
    return prediction

  def export_dataset(self):
    """
    Exports dataset as .csv file
    """
    
    self.labeled_dataset.to_csv("number_array_dataset.csv",index=False)

#### Make object of class

This line of code creates an Object of the class and all methods are present in that class.

In [ ]:
# NumberArray ( n, k )
# First argument is N ( Maximum number in the Array ) and second argument is K ( Length of the array ).

number_array = NumberArray(10, 10)

#### Create dataset using create_dataset() method

We discourage using this method as there may be very large amount of permutations and you may get memory error due to memory constraints.

Use create_random_sample_dataset(no_of_sample) instead. It takes no of samples as argument and create dataset.

In [ ]:
#number_array.create_dataset()


### Create Dataset with Random Samples of the permutation

In [ ]:
number_array.create_random_sample_dataset(100000)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,1,2,3,4,5,6,7,8,9,10
0,7,9,9,2,5,8,3,1,8,7
1,8,4,7,5,5,1,6,10,5,1
2,3,2,2,7,6,9,3,6,8,2
3,5,7,5,2,5,2,2,7,7,2
4,1,5,7,10,5,10,4,6,5,4
...,...,...,...,...,...,...,...,...,...,...
99995,5,2,1,2,7,9,10,5,2,5
99996,4,10,4,7,6,8,4,5,2,9
99997,9,3,10,9,8,3,10,2,10,8
99998,10,1,5,1,2,2,1,5,4,8


#### Label dataset and create new features using feature engineering 

In [ ]:
label_df = number_array.label_dataset()

In [ ]:
label_df

,1,2,3,4,5,6,7,8,9,10,unique,difference_max,difference_last,output
0,7,9,9,2,5,8,3,1,8,7,7,8,0,20
1,8,4,7,5,5,1,6,10,5,1,7,9,2,20
2,3,2,2,7,6,9,3,6,8,2,6,7,1,20
3,5,7,5,2,5,2,2,7,7,2,3,5,0,20
4,1,5,7,10,5,10,4,6,5,4,6,9,0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,5,2,1,2,7,9,10,5,2,5,6,9,1,20
99996,4,10,4,7,6,8,4,5,2,9,8,8,1,20
99997,9,3,10,9,8,3,10,2,10,8,5,8,0,20
99998,10,1,5,1,2,2,1,5,4,8,6,9,2,20


In [ ]:
print("Occurence of 10 => ",label_df[label_df['output'] == 10].shape[0])
print("Occurence of 20 => ",label_df[label_df['output'] == 20].shape[0])
print("Occurence of 30 => ",label_df[label_df['output'] == 30].shape[0])

Occurence of 10 =>  51
Occurence of 20 =>  98733
Occurence of 30 =>  1216


### Train Model

In [ ]:
df = number_array.train_model(test_size=0.3)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Number of mislabeled points out of a total 88860 points : 4
Accuracy of the Output is: 99.99549853702453%


In [ ]:
df

,1,2,3,4,5,6,7,8,9,10,unique,difference_max,difference_last,Original Output,Predicted Output
0,6,2,6,4,2,3,4,5,7,4,6,5,0,30,30
1,10,6,2,7,6,8,1,9,4,7,8,9,1,20,20
2,2,1,1,1,1,1,1,1,9,1,3,8,7,10,10
3,9,2,1,1,1,1,1,1,1,1,3,8,7,10,10
4,1,1,1,2,2,2,1,10,2,1,3,9,8,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88855,6,5,5,6,3,3,4,3,4,3,4,3,1,30,30
88856,6,3,4,3,5,7,5,4,5,7,5,4,0,30,30
88857,5,7,7,5,6,4,4,5,6,3,6,5,1,30,30
88858,10,8,7,7,9,3,1,9,9,10,6,9,0,20,20


In [ ]:
print("Original Occurences")
print("Occurence of 10 => ",df[df['Original Output'] == 10].shape[0])
print("Occurence of 20 => ",df[df['Original Output'] == 20].shape[0])
print("Occurence of 30 => ",df[df['Original Output'] == 30].shape[0])

print("\nY Predict Occurences")
print("Occurence of 10 => ",df[df['Predicted Output'] == 10].shape[0])
print("Occurence of 20 => ",df[df['Predicted Output'] == 20].shape[0])
print("Occurence of 30 => ",df[df['Predicted Output'] == 30].shape[0])

Original Occurences
Occurence of 10 =>  29711
Occurence of 20 =>  29592
Occurence of 30 =>  29557

Y Predict Occurences
Occurence of 10 =>  29711
Occurence of 20 =>  29590
Occurence of 30 =>  29559


### Predict Sample using model

This method could be used for predicting the output of a single sample supplied.

In [ ]:
number_array.predict_sample([10,1,1,1,1,1,1,1,1,1])

array([10])

#### Export Dataset as .csv file

This method exports the dataset as a .csv file

In [ ]:
number_array.export_dataset()